Universidad Torcuato Di Tella

Licenciatura en Tecnología Digital\
**Tecnología Digital VI: Inteligencia Artificial**


In [ ]:
!pip install torchaudio
!pip install  pydub
!pip install wandb
import os
import torch
import torchaudio
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchaudio.datasets import GTZAN
from torch.utils.data import DataLoader
import torchaudio.transforms as tt
from torch.utils.data import random_split
import matplotlib
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
import torchaudio.transforms as transforms
%matplotlib inline



# TP3: Encodeador de música



## Orden de pasos

0. Elijan GPU para que corra mas rapido (RAM --> change runtime type --> T4 GPU)
1. Descargamos el dataset y lo descomprimimos en alguna carpeta en nuestro drive.
2. Conectamos la notebook a gdrive y seteamos data_dir con el path a los archivos.
3. Visualización de los archivos
4. Clasificación
5. Evaluación




In [ ]:
project_name='Music_genre_classification'

### 2. Conectamos la notebook a gdrive y seteamos data_dir con el path a los archivos.





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


data_dir es el path donde pusimos la carpeta genres. "'//content/drive/MyDrive/Materias/TD6 - Inteligencia Artificial/TPs/2023/TP4/genres/'" es un ejemplo. Modificar.

In [ ]:
import os
data_dir='//content/drive/MyDrive/TP3 TD6/genres_5sec/'
list_files=os.listdir(data_dir)
classes=[]
for file in list_files:
  name='{}/{}'.format(data_dir,file)
  if os.path.isdir(name):
    classes.append(file)
classes

['blues',
 'rock',
 'hiphop',
 'reggae',
 'country',
 'disco',
 'jazz',
 'classical',
 'metal',
 'pop']

In [ ]:
samplerate=22050
def parse_genres(fname):
    parts = fname.split('/')[-1].split('.')[0]
    return parts #' '.join(parts[0])

class MusicDataset(Dataset):
    def __init__(self, root):
        super().__init__()
        self.root = root
        self.files =[]
        for c in classes:
          self.files = self.files + [fname for fname in os.listdir(os.path.join(root,c)) if fname.endswith('.wav')]
        self.classes = list(set(parse_genres(fname) for fname in self.files))
        #self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        fname = self.files[i]

        #img = self.transform(open_image(fpath))
        genre = parse_genres(fname)
        fpath = os.path.join(self.root,genre, fname)
        class_idx = self.classes.index(genre)
        audio = torchaudio.load(fpath)[0].squeeze()

        return audio, class_idx
dataset = MusicDataset(data_dir)

### 3. Visualización de los archivos

In [ ]:
waveform,label= dataset[0]

In [ ]:
random_seed = 42
torch.manual_seed(random_seed);
val_size = 100
test_size = 100
train_size = len(dataset) - val_size - test_size

train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])
len(train_ds),len(val_ds),len(test_ds)

(790, 100, 100)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 20

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
valid_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
test_dl = DataLoader(test_ds,1, num_workers=4, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


### 4. Clasificación

In [ ]:
class CNN_4(nn.Module):
    def __init__(self):
        super().__init__()

        # Definir la transformación MelSpectrogram
        self.spectrogram_transform = transforms.MelSpectrogram(
            sample_rate=22050,
            n_fft=1024,
            hop_length=512,
            n_mels=64
        )

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(27648, 32)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 10)

    def forward(self, x):
        # Convertir waveform a espectrograma
        batch_spectrograms = [self.spectrogram_transform(wav.squeeze(0)) for wav in x]
        x = torch.stack(batch_spectrograms).unsqueeze(1).to(x.device)  # Agrega el canal

        x = nn.Tanh()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)

        x = nn.Tanh()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)

        x = torch.flatten(x, 1)
        x = nn.Tanh()(self.fc1(x))
        x = self.dropout(x)
        x = nn.Tanh()(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = CNN_4()
model.to(device)
print(model)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n = count_parameters(model)
print("Number of parameters: %s" % n)

CNN_4(
  (spectrogram_transform): MelSpectrogram(
    (spectrogram): Spectrogram()
    (mel_scale): MelScale()
  )
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=27648, out_features=32, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)
Number of parameters: 895722


In [ ]:
project_name='Modelo grupo Bedacarratz Gaido Kullock'
wandb.login(key = "903378b08bbf4bb7597553b9faa17b1eb2bacc01")
wandb.init(
    project=project_name,
    entity="",
    name="mejor modelo",
    config = {
        "learning_rate": 0.01,
        "epochs": 10
    }
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: estanibeda (estanibeda-universidad-torcuato-di-tella). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Este codigo es para CNN


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import gc
import numpy as np
import wandb
from sklearn.metrics import accuracy_score, confusion_matrix
import torch.nn.functional as F
import torchaudio.transforms as transforms

# Configuración
loss_function = nn.CrossEntropyLoss()
valid_losses = []
train_losses = []
num_epochs = 30
torch.cuda.empty_cache()
gc.collect()

# Versión 2: Adam + StepLR
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5) #
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

l1_lambda = 1e-4


# Loop de entrenamiento
for epoch in range(num_epochs):
    model.train()
    train_losses = []  # Reseteamos los train_losses para cada época

    # Entrenamiento
    for wav, genre_index in train_dl:
        optimizer.zero_grad()

        genre_index = genre_index.to(device)

        # Forward y cálculo de pérdida
        out = model(wav)
        loss = loss_function(out.squeeze(), genre_index)

        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss += l1_lambda * l1_norm  # Agrega la penalización L1 a la pérdida

        # Backward y actualización de optimizador
        loss.backward()
        optimizer.step()

        # Almacenar pérdida
        train_losses.append(loss.item())

        # Liberar memoria
        del wav, genre_index, loss, out
        torch.cuda.empty_cache()
        gc.collect()

    # Cálculo de pérdida de entrenamiento de la época
    train_loss_epoch = sum(train_losses) / len(train_losses)
    print(f'Epoch: [{epoch + 1}/{num_epochs}], Train loss: {train_loss_epoch:.4f}')

    # Validación
    model.eval()
    valid_losses_epoch = []
    correct = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for wav, genre_index in valid_dl:
            wav = wav.to(device)
            genre_index = genre_index.to(device)

            # Forward y cálculo de pérdida de validación
            out = model(wav)
            loss = loss_function(out.squeeze(), genre_index)

            # Almacenar pérdida y realizar predicciones
            valid_losses_epoch.append(loss.item())
            pred = out.argmax(dim=-1).flatten()
            correct += pred.eq(genre_index).sum().item()
            y_true.extend(genre_index.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())

            # Liberar memoria
            del wav, genre_index, loss, out
            torch.cuda.empty_cache()
            gc.collect()

    # Cálculo de métricas de validación
    valid_loss_epoch = sum(valid_losses_epoch) / len(valid_losses_epoch)
    accuracy = correct / len(valid_dl.dataset)
    print(f'Epoch: [{epoch + 1}/{num_epochs}], Valid loss: {valid_loss_epoch:.4f}, Valid accuracy: {accuracy:.4f}')

    # Guardar en WandB
    wandb.log({"Train Loss": train_loss_epoch, "Valid Loss": valid_loss_epoch, "Valid Accuracy": accuracy})

    # Guardar mejor modelo
    valid_losses.append(valid_loss_epoch)
    if np.argmin(valid_losses) == epoch:
        print(f'Saving the best model at epoch {epoch}!')
        torch.save(model.state_dict(), 'best_model.ckpt')

    scheduler.step()

wandb.finish()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch: [1/30], Train loss: 2.5721
Epoch: [1/30], Valid loss: 2.1962, Valid accuracy: 0.2500
Saving the best model at epoch 0!
Epoch: [2/30], Train loss: 2.3811
Epoch: [2/30], Valid loss: 2.0194, Valid accuracy: 0.3700
Saving the best model at epoch 1!
Epoch: [3/30], Train loss: 2.2512
Epoch: [3/30], Valid loss: 1.8476, Valid accuracy: 0.4500
Saving the best model at epoch 2!
Epoch: [4/30], Train loss: 2.0916
Epoch: [4/30], Valid loss: 1.7077, Valid accuracy: 0.5400
Saving the best model at epoch 3!
Epoch: [5/30], Train loss: 1.9018
Epoch: [5/30], Valid loss: 1.6165, Valid accuracy: 0.4900
Saving the best model at epoch 4!
Epoch: [6/30], Train loss: 1.7008
Epoch: [6/30], Valid loss: 1.5473, Valid accuracy: 0.5300
Saving the best model at epoch 5!
Epoch: [7/30], Train loss: 1.5511
Epoch: [7/30], Valid loss: 1.4719, Valid accuracy: 0.5700
Saving the best model at epoch 6!
Epoch: [8/30], Train loss: 1.4158
Epoch: [8/30], Valid loss: 1.4143, Valid accuracy: 0.6200
Saving the best model at e

Train Loss,█▇▇▆▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Valid Accuracy,▁▃▅▆▆▆▇█▇▇▇▇██▇████▇█▇▇▇█▇████
Valid Loss,█▇▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,0.69478
Valid Accuracy,0.61
Valid Loss,1.24863




### 5. Evaluación



In [ ]:
test_dl = DataLoader(test_ds,1,shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
# Load the best model
S = torch.load('best_model.ckpt')
model.load_state_dict(S)
print('loaded!')

# Run evaluation
model.eval()
y_true = []
y_pred = []
correct = 0


with torch.no_grad():
    for wav, genre_index in test_dl:
        wav = wav.to(device)
        genre_index = genre_index.to(device)

        out = model(wav)

        pred= out.argmax(dim=-1).flatten()
        # append labels and predictions
        correct += pred.eq(genre_index).sum().item()
        y_true.extend(genre_index.cpu().numpy())
        y_pred.extend(pred.cpu().numpy())

accuracy =correct/ len(test_dl.dataset)
print('Epoch: [%d/%d], Valid loss: %.4f, Valid accuracy: %.4f' % (epoch+1, num_epochs, train_loss_epoch, accuracy))


loaded!


<ipython-input-16-f502dbe71b99>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  S = torch.load('best_model.ckpt')


Epoch: [30/30], Valid loss: 0.6948, Valid accuracy: 0.6800


In [ ]:
waveform,label= test_dl.dataset[12]
print("shape of waveform {}, sample rate with {}, label is {} ".format(waveform.size(),samplerate,label))

In [ ]:
import IPython
IPython.display.Audio(waveform, rate=22050)

In [ ]:
wav= torch.unsqueeze(waveform, dim=0)
model.to(device)
wav =wav.to(device)
out = model(wav)
pred= out.argmax(dim=-1).flatten()
classes[pred], classes[label]